# Loading images
### Section 2.2 of the Supplementary Information (Figure 5 and Table 4)

In [ ]:
import numpy as np
import matplotlib.image as image

In [ ]:
# Image load.
matrix_dim = 128

img=image.imread(f'data/zebra_{matrix_dim}.bmp')
vec=img.reshape(-1)
input_state = vec/np.linalg.norm(vec)

In [ ]:
baa_strategy = 'greedy'
baa_low_rank = False

In [ ]:
# Plots examples of the states that represent the images.

_dpi = 96
def plot_digits(vectors, labels, text=''):
    import matplotlib.pyplot as plt
    ncols = len(vectors)
    _, axes = plt.subplots(nrows=1, ncols=ncols, figsize=(ncols*1.2*matrix_dim/_dpi, 1.2*matrix_dim/_dpi), dpi=_dpi)
    for ax, vector, label in zip(axes, vectors, labels):
        ax.set_axis_off()
        image = vector[:int(matrix_dim**2)].reshape(matrix_dim, matrix_dim)
        ax.imshow(image, cmap=plt.cm.gray, interpolation='none')
        ax.set_title(label)

    plt.savefig(f'save/zebra{text}.pdf')
    plt.show()

# Ideal

In [ ]:
from qclib.state_preparation.util.baa import adaptive_approximation

_images = []
_labels = []
for l_max in [i/100 for i in range(11)]:
    print(l_max, end=' ')
    node = adaptive_approximation(input_state, max_fidelity_loss=l_max, strategy=baa_strategy, use_low_rank=baa_low_rank)
    _images.append(node.state_vector()**2)
    _labels.append(str(l_max))

In [ ]:
plot_digits(_images, _labels, '_ideal')

# Noisy

In [ ]:
from qiskit import IBMQ, ClassicalRegister, execute, transpile
from qiskit.tools.monitor import job_monitor
from qiskit.providers.aer.noise import NoiseModel
from qiskit.test.mock import FakeMumbai

from qclib.state_preparation.baa_schmidt import initialize as baa_schmidt

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-research') # 'ibm-q'

In [ ]:
backend  = provider.get_backend('ibmq_qasm_simulator')
noisy_backend  = FakeMumbai()

shots      = 8192

noise_model = None
coupling_map = None
basis_gates = None

''' uncomment to include noise in the simulation '''
#noise_model = NoiseModel.from_backend(noisy_backend)
#coupling_map = noisy_backend.configuration().coupling_map
#basis_gates = noise_model.basis_gates

In [ ]:
def measurement(job):
    counts = job.result().get_counts()
    v = sum(counts.values())
    
    counts2 = {}
    for m in range(2**n):
        pattern = '{:0{}b}'.format(m, n)
        if pattern in counts:
            counts2[pattern] = counts[pattern]
        else:
            counts2[pattern] = 0.0

    prob = { key : value/v for (key, value) in counts2.items() }

    return np.array([val for key, val in prob.items()])    

In [ ]:
jobs = {}
circuits = {}
t_circuits = {}

In [ ]:
start = 0
end = 11

In [ ]:
# Build the quantum circuits
for l_max in [i/100 for i in range(start, end)]:
    print(l_max, ' ', end='')
    circuit = baa_schmidt(input_state, max_fidelity_loss=l_max, strategy=baa_strategy, use_low_rank=baa_low_rank)
    circuit.measure_all()
    circuits[l_max] = circuit

In [ ]:
# Transpile the quantum circuits and count the number of CNOTs
for l_max in [i/100 for i in range(start, end)]:
    print(l_max, ' ', end='')
    circuit = circuits[l_max]

    t_circuit = transpile(circuit, basis_gates=['u1','u2','u3', 'cx'])
    ops = t_circuit.count_ops()
    print(ops['cx'], ' ', end='')

    t_circuits[l_max] = t_circuit

In [ ]:
for l_max in [i/100 for i in range(start, end)]:
    jobs[l_max] = []

In [ ]:
import time
from qiskit.providers.jobstatus import JobStatus

def count_running_jobs(jobs):
    count = 0
    for _, jobs_list in jobs.items():
        count += sum([job.status() in (JobStatus.RUNNING, JobStatus.INITIALIZING, JobStatus.QUEUED, JobStatus.VALIDATING) for job in jobs_list])
    return count

reps = 20
print('start', start, 'end',end, 'reps', reps)
for l_max in [i/100 for i in range(start, end)]:
    print(l_max, ' ', end='')

    for rep in range(reps):
        print('.', end='')
        while count_running_jobs(jobs) >= 20: # maximum number (20) of concurrent jobs
            time.sleep(5) # Sleep for 5 seconds

        job = execute(t_circuits[l_max], backend, shots=shots, 
                        coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model)
        jobs[l_max].append(job)
        

In [ ]:
_images = []
_labels = []
for l_max in [i/100 for i in range(start, end)]:
    print(l_max, end=' ')
    print(len(jobs[l_max]), end='')
    results = []
    for job in jobs[l_max]:
        print('.', end='')
        job_monitor(job, quiet=True)
        try:
            results.append(measurement(job))
        except Exception as e:
            print(e)

    _images.append(np.mean(results, axis=0))
    _labels.append(str(l_max))

In [ ]:
plot_digits(_images, _labels, '_'+backend.name())